# ACTIVIDAD 3 CASO: EVALUACIÓN DE LA SEGMENTACIÓN

Realizado por:
* Nicolás Felipe Trujillo Montero
* Jesús Carlos Avecilla de la Herran
* Rubén González Navarro

## Primera Parte

En primer lugar, se propone intentar desarrollar un algoritmo que, dada una imagen donde se encuentren una o varias aves volando, sea capaz de segmentar dicha ave o aves.

Se desarrolla un algoritmo que usa el algoritmo de KMeans para realizar la segmentación por colores de una imagen dada como input. En este caso, como ejemplo se van a usar la siguientes imágenes:

![Primera imagen usada para la segmentación](img_birds_1.png 'Primera imagen usada para la segmentación')


![Segunda imagen usada para la segmentación](img_birds_2.png 'Segunda imagen usada para la segmentación')

In [3]:
######################################################################
# Librerias usadas 
######################################################################
import random
import cv2
import numpy as np
from tqdm import tqdm
from skimage import data, color
from skimage import io
import matplotlib.pyplot as plt
import seaborn as sns

El algoritmo constará de los siguientes pasos:
* Lectura de la imagen.
* Transformación de los píxeles de la imagen a un tipo de dato float de 32 bits.
* Definición del algoritmo:
    * Para el algoritmo de KMedias deberemos definir la variable K, la cual indicará el número de clusters de agruparción. No obstante, se probará con varios valores de K y para ello creamos un bucle for que vaya incrementando esta variable.
    * Definimos también el criterio de parada del algoritmo. Para ello, se hará uso de variables predefinidas por la librería, además de definir el número máximo de iteraciones del algoritmo y de la precisión deseada.
    * Aplicación del algoritmo, el cual nos devolverá dos arrays. El primero contendrá todas las etiquetas de los clusters encontrados y el segundo los centros de estos.
    * Transformamos el array de etiquetas a un tipo de dato entero de 8 bits.
    * Guardamos el resultado concatenando el array de centros y de etiquetas.
* Guardamos a disco la imagen segmentada, pero además iremos desagrupondo cada cluster para visualizar lo que se ha obtenido en una máscara independiente.

In [7]:
# Lectura de la imagen
prefix = 'img1'
file = prefix + '.jpg'
img = cv2.imread(file)

# Cambio a tipo de dato float de 32 bits
Z = np.float32(img.reshape((-1,3)))

# Definición del algoritmo KMeans
for K in range(1,11):
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    _,labels,centers = cv2.kmeans(Z, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    labels = labels.reshape((img.shape[:-1]))
    reduced = np.uint8(centers)[labels]

    result = [np.hstack([img, reduced])]
    for i, c in enumerate(centers):
        mask = cv2.inRange(labels, i, i)
        mask = np.dstack([mask]*3) # Conversión a máscara de 3 canales
        ex_img = cv2.bitwise_and(img, mask)
        ex_reduced = cv2.bitwise_and(reduced, mask)
        result.append(np.hstack([ex_img, ex_reduced]))

    cv2.imwrite('{}_K{}.jpg'.format(prefix,K), np.vstack(result))

Habiendo procesado la primera imagen, obtenemos que para una valor de K igual a 3 ya es posible segmentar las aves de la imagen. No obstante, la precisión de la segmentación que deseeamos dependerá de la aplicaicón final del algoritmo. Además convendría aplicar otras técnicas de procesamiento de imagen para mejorar la máscara obtenida, en concreto se podría proponer usar una serie de operaciones morfológicas dado que se quedan algunos huecos en la máscara de las aves segmentadas. 

Destacar también que para valores de K mayores a 5 se empiezan a obtener en cierta medida los contornos de las aves, que como se ha comentado antes según la aplicación puede ser interesante según la aplicación y también se podrían usar tras aplicarle ciertas técnicas, como pueden ser operaciones morfológicas, rellenado de huecos (*cv2.imfill*), cálculo del áerea, cerco convexo y aplicar posteriormente incluso una correlación cruzada en otras imagenes para sacar otro tipo de estadísticas por ejemplo.

Se muestra a continuación el resultado obtenido para K = 3 y para K = 10. Los de más archivos se encuentran en la carpeta de la entrega.

![Resultado de la primera imagen con K igual a 3](img_birds_1_K3.jpg "Resultado de la primera imagen con K igual a 3")

![Resultado de la primera imagen con K igual a 10](img_birds_1_K10.jpg "Resultado de la primera imagen con K igual a 10")

En las mismas condiciones, mostramos los mismos resultados para la segunda imagen.

![Resultado de la segunda imagen con K igual a 3](img_birds_2_K3.jpg "Resultado de la segunda imagen con K igual a 3")

![Resultado de la segunda imagen con K igual a 10](img_birds_2_K10.jpg "Resultado de la segunda imagen con K igual a 10")

## Segunda Parte

Llegados a este punto del trabajo, se intentó de igual forma aplicar este algoritmo a otro caso de estudio y ver su funcionamiento. Se propuso resolver el famoso juego de Where's Wally? aplicando el mismo algoritmo usado anteriormente y poder de este modo encontrar a Wally. Para ello se hizo uso de las siguientes imágenes:

![Primera imagen Where's Wally?](img1.jpg "Primera imagen Where's Wally?")

![Segunda imagen Where's Wally?](img2.jpg "Segunda imagen Where's Wally?")

Decidimos intentar resolverlo mediante la segmentación de colores dado que al ser Wally un personaje vestido de rojo y blanco y estar camuflado cerca de colores totalmente diferentes al suyo, sería fácil localizarlo. No obstante, no se consiguió resolver el problema como esperábamos y quizás abría que hacer ciertas modificaciones al presente algoritmo o usar otro segmentador. Otros segmentadores que se intentaron utilizar fueron los siguientes:

* Ade20k Model.
* Mapillary Model.
* COCO Model.
* MSeg Model.

Sin embargo, todos ellos eran demasiado pesados y ningún miembro del grupo consiguió descargarlos correctamente para probar su funcionamiento con estas imágenes.

Mostramos a continuación los resultados obtenidos para la primera imagen con valores de K = 4 y K = 8, teniendo en la carpeta de la entrega las demás imágenes de igual modo que las de la primera parte.

![Resultado de la primera imagen Where's Wally? con K igual a 4](img1_K4.jpg "Resultado de la primera imagen Where's Wally? con K igual a 4")

![Resultado de la primera imagen Where's Wally? con K igual a 8](img1_K8.jpg "Resultado de la primera imagen Where's Wally? con K igual a 8")

Mostramos a continuación los resultados obtenidos para la segunda imagen con los mismos valores de K.

![Resultado de la segunda imagen Where's Wally? con K igual a 4](img2_K4.jpg "Resultado de la segunda imagen Where's Wally? con K igual a 4")

![Resultado de la segunda imagen Where's Wally? con K igual a 8](img2_K8.jpg "Resultado de la segunda imagen Where's Wally? con K igual a 8")

## Bibliografía

* Documentación algoritmo KMeans de OpenCV 3.X.X: https://docs.opencv.org/3.4/d1/d5c/tutorial_py_kmeans_opencv.html
